<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [162]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [163]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
Using competition: 26jun23
  teamId  teamName                       submissionDate       score    
--------  -----------------------------  -------------------  -------  
10612701  Никита Угрюмов                 2023-06-30 11:19:47  0.88394  
10612695  Ne1tr0N [Maxim Mashtaler]      2023-06-29 22:06:01  0.88242  
10612706  Michael Kondrashov             2023-06-29 22:56:06  0.88231  
10616508  Ulyana Klushova                2023-06-29 22:43:08  0.87691  
10614521  Konstantin Zeigman             2023-06-30 11:19:20  0.87691  
10612715  Stepan Vysotsky                2023-06-29 19:30:11  0.87687  
10612698  Alexandr Boldirev              2023-06-30 10:43:59  0.87539  
10617156  Gleb Goloshchapov              2023-06-30 11:08:02  0.87355  
10612681  Maria Alyushina                2023-06-29 18:43:03  0.87097  
10612680  esheshka                       2023-06-29 14:43:48  0.86811  
10614699  Dmitr

In [164]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 1.48 s, sys: 81.8 ms, total: 1.56 s
Wall time: 34 s


In [165]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC

from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.model_selection import GridSearchCV as GSCV

In [166]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc')

CPU times: user 7.04 ms, sys: 0 ns, total: 7.04 ms
Wall time: 17.8 ms


In [167]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc');

CPU times: user 5.62 ms, sys: 0 ns, total: 5.62 ms
Wall time: 15.7 ms


In [168]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid')
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')

CPU times: user 6.95 ms, sys: 43 µs, total: 6.99 ms
Wall time: 15.7 ms


In [169]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm'])

CPU times: user 9.67 s, sys: 780 ms, total: 10.4 s
Wall time: 22.8 s


In [170]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 4)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


⏳ started. You have 120 sec. Good luck!


# **Imports**

In [171]:
!pip install lightgbm
!pip install catboost

from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

# **Interesting**

In [172]:
df_test = dfTrx.join(gender, on='cid', how='inner')
df_test = df_test[~df_test.gender.isna()]

In [173]:
#@title MCC

mcc_list = list(set(df_test.mcc))
df_mcc = pd.DataFrame({'mcc': mcc_list})

zero = df_test[df_test.gender == 0][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
zero = df_mcc.join(pd.DataFrame({'zero': zero}), on='mcc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
one = df_mcc.join(pd.DataFrame({'one': one}), on='mcc', how='outer').fillna(0)

df_mcc = df_mcc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='mcc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='mcc', how='left')

interesting_mcc = set(df_mcc[(df_mcc['count'] > 100) & ((df_mcc['%'] > 70) | (df_mcc['%'] < 30))].mcc)

interesting_mcc

{62, 63, 70, 71, 82, 106, 149, 150, 161}

In [174]:
#@title TTC

ttc_list = list(set(df_test.ttc))
df_ttc = pd.DataFrame({'ttc': ttc_list})

zero = df_test[df_test.gender == 0][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
zero = df_ttc.join(pd.DataFrame({'zero': zero}), on='ttc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
one = df_ttc.join(pd.DataFrame({'one': one}), on='ttc', how='outer').fillna(0)

df_ttc = df_ttc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='ttc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='ttc', how='left')

interesting_ttc = set(df_ttc[(df_ttc['count'] > 100) & ((df_ttc['%'] > 70) | (df_ttc['%'] < 30))].ttc)

interesting_ttc

set()

In [175]:
#@title TID

tid_list = list(set(df_test.tid))
df_tid = pd.DataFrame({'tid': tid_list})

zero = df_test[df_test.gender == 0][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
zero = df_tid.join(pd.DataFrame({'gender': zero}), on='tid', how='outer').fillna(0).astype({'gender': int})
one = df_test[df_test.gender == 1][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
one = df_tid.join(pd.DataFrame({'gender': one}), on='tid', how='outer').fillna(0).astype({'gender': int})

df_tid = df_tid
df_tid = df_tid.join(zero[['gender']] + one[['gender']]).rename(columns={'gender': 'count'}).join(100 * zero[['gender']] / (zero[['gender']] + one[['gender']])).rename(columns={'gender': '%'})

interesting_tid = set(df_tid[(df_tid['count'] > 100) & ((df_tid['%'] > 70) | (df_tid['%'] < 30))].tid)

interesting_tid

{'10244893', '20001004', '20001033', '20828301', '940047'}

# **My Code**

In [176]:
#@title Настройки

large_count = 2000
large_sum = -1000000000

k = 1
k1 = 0.8
k2 = 0.85
# k3 = 1.25
k3 = 8

In [177]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count * k1].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum * k1].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 1.13 s, sys: 74 ms, total: 1.2 s
Wall time: 2.71 s


36

In [178]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count * k2].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum * k2) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 1.02 s, sys: 81 ms, total: 1.1 s
Wall time: 2.54 s


25

In [179]:
#@title Определение наиболее используемых tid
%%time
tid_by_cid = dfTrx[['cid', 'tid']].drop_duplicates().astype({'tid': str}).groupby('cid').agg({'tid': ' '.join}).sort_values('cid')

large_tid_count = dfTrx[['cid', 'tid']].drop_duplicates().groupby('tid').agg({'cid': 'size'}).sort_values(by = 'cid')
large_tid_sum = dfTrx[['amt', 'tid']].groupby('tid').agg('sum').sort_values(by = 'amt')

most_popular_tid = set(large_tid_count[large_tid_count.cid > large_count * k3].index.to_list()) | set(large_tid_sum[(large_tid_sum.amt < large_sum * k3) | (large_tid_sum.amt > 1000000000)].index.to_list())
len(most_popular_tid)

CPU times: user 6.3 s, sys: 138 ms, total: 6.44 s
Wall time: 9.65 s


0

# **Начало создания df по cid**

In [180]:
#@title Сумма

def s(to_df):
    return to_df / to_df.sum(axis=0)
    # return to_df / np.sqrt((to_df**2).sum(axis=0))
    # return to_df / (np.sqrt(to_df**2)).sum(axis=0)

In [181]:
#@title Настройки

stat = ['count', 'sum', 'median']

In [182]:
%%time
df = df_.copy(deep=True)

CPU times: user 1.96 ms, sys: 7 µs, total: 1.97 ms
Wall time: 12.1 ms


In [183]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

for mcc_index in (most_popular_mcc - interesting_mcc):
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc or interesting_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for mcc_index in interesting_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

to_df = s(to_df) * 200
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 2.17 s, sys: 5.57 ms, total: 2.18 s
Wall time: 2.99 s


In [184]:
#@title TTC
to_df = df_.copy(deep=True)

for ttc_index in (most_popular_ttc - interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc or interesting_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for ttc_index in (interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [185]:
#@title AMT

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [186]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(0, 1):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df) * 2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(1, 9):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

# to_df = df_.copy(deep=True)
# for hour in range(9, 24):
#     stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
#     to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

# to_df = s(to_df)
# df = df.join(to_df, on = 'cid', how='left')

CPU times: user 5.59 s, sys: 75 ms, total: 5.67 s
Wall time: 10.1 s


In [187]:
#@title Date
%%time
to_df = df_.copy(deep=True)

# for weekday in [0, 3, 4, 5, 6]:
#     stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
#     to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 345 ms, sys: 909 µs, total: 346 ms
Wall time: 349 ms


In [188]:
#@title TID
to_df = df_.copy(deep=True)

for tid_index in (most_popular_tid - interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

stat_by_other_tid = dfTrx[~dfTrx.tid.isin(most_popular_tid or interesting_tid)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_tid.columns += '_tid'
to_df = to_df.join(stat_by_other_tid, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for tid_index in (interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [189]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

In [210]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.28, random_state=0) # split train observation into local train/test sets (to score performance)


CPU times: user 33.9 ms, sys: 978 µs, total: 34.9 ms
Wall time: 138 ms


In [ ]:
model = CBC(iterations=200)
params = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.02, 0.03],
    'loss_function' : ['Logloss'],
    'eval_metric' : ['AUC'],
    'boosting_type' : ['Plain'],
    'bootstrap_type' : ['MVS'],
}

model = GSCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)

clf = model.fit(trainX0, trainY0.gender)
print(model.best_params_)
pY = clf.predict_proba(testX0)[:, 1]

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))
print(pY.sum() / pY.shape[0])

# **Обучение реальное**

In [214]:
model_final = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)

model1 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
model2 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model3 = CBC(iterations=200)
model4 = LGBMC()

clf1 = model1.fit(trainX0, trainY0.gender)
clf2 = model2.fit(trainX0, trainY0.gender)
clf3 = model3.fit(trainX0, trainY0.gender)
clf4 = model4.fit(trainX0, trainY0.gender)

pY1_r = clf1.predict_proba(testX0)[:, 1]
pY2_r = clf2.predict_proba(testX0)[:, 1]
pY3_r = clf3.predict_proba(testX0)[:, 1]
pY4_r = clf4.predict_proba(testX0)[:, 1]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.08653
0:	learn: 0.6705910	total: 83.4ms	remaining: 16.6s
1:	learn: 0.6500710	total: 139ms	remaining: 13.7s
2:	learn: 0.6326781	total: 211ms	remaining: 13.8s
3:	learn: 0.6156782	total: 267ms	remaining: 13.1s
4:	learn: 0.6036694	total: 319ms	remaining: 12.4s
5:	learn: 0.5937613	total: 370ms	remaining: 12s
6:	learn: 0.5837500	total: 423ms	remaining: 11.7s
7:	learn: 0.5716356	total: 486ms	remaining: 11.7s
8:	learn: 0.5640891	total: 537ms	remaining: 11.4s
9:	learn: 0.5550314	total: 589ms	remaining: 11.2s
10:	learn: 0.5475275	total: 643ms	remaining: 11s
11:	learn: 0.5407268	total: 693ms	remaining: 10.9s
12:	learn: 0.5346633	total: 750ms	remaining: 10.8s
13:	learn: 0.5289591	total: 804ms	remaining: 10.7s
14:	learn: 0.5236457	total: 859ms	remaining: 10.6s
15:	learn: 0.5187181	total: 911ms	remaining: 10.5s
16:	learn: 0.5136110	total: 962ms	remaining: 10.4s
17:	learn: 0.5093338	total: 1.01s	remaining: 10.3s
18:	learn: 0.5056604	total: 1.08s	remaining: 10.3s
19:	learn: 0.50

In [215]:
pY = (pY1_r + 2*pY2_r + 4*pY3_r + 2*pY4_r) / 9

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))
print(pY.sum() / pY.shape[0])

Local AUC ROC 0.8682387554612723
0.432935483217705


In [ ]:
# pY = pd.DataFrame(pY, columns=['gender'])
# pY.index.name = 'cid'
# print(pY.sum() / pY.shape[0])

In [ ]:
# pY.reset_index().to_csv('💳Baseline.csv', index_label='cid', index=False, float_format='%10f')

In [216]:
X_t_r = np.array([pY1_r, pY2_r, pY3_r, pY4_r]).T

p_t_r = model_final.fit(X_t_r, testY0.gender)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [217]:
pY1_f = clf1.predict_proba(testX)[:, 1]
pY2_f = clf2.predict_proba(testX)[:, 1]
pY3_f = clf3.predict_proba(testX)[:, 1]
pY4_f = clf4.predict_proba(testX)[:, 1]

X_t_f = np.array([pY1_f, pY2_f, pY3_f, pY4_f]).T

pY = model_final.predict_proba(X_t_f)[:, 1]

In [218]:
pY = pd.DataFrame(pY, columns=['gender'])
pY.index.name = 'cid'
print(pY.sum() / pY.shape[0])

gender    0.443959
dtype: float64


In [ ]:
pY

In [ ]:
pY.reset_index().to_csv('💳Baseline.csv', index_label='cid', index=False, float_format='%10f')

In [ ]:
tmr.ShowTime()

# **Обучение пробное**

In [207]:
model_final = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)

model1 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
model2 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model3 = CBC(iterations=200)
model4 = LGBMC()

pY1 = model1.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY2 = model2.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY3 = model3.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY4 = model4.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.080057
0:	learn: 0.6736480	total: 122ms	remaining: 24.2s
1:	learn: 0.6570934	total: 225ms	remaining: 22.3s
2:	learn: 0.6398242	total: 324ms	remaining: 21.3s
3:	learn: 0.6247399	total: 424ms	remaining: 20.8s
4:	learn: 0.6146346	total: 522ms	remaining: 20.3s
5:	learn: 0.6030220	total: 617ms	remaining: 20s
6:	learn: 0.5907800	total: 722ms	remaining: 19.9s
7:	learn: 0.5815643	total: 823ms	remaining: 19.8s
8:	learn: 0.5725044	total: 921ms	remaining: 19.5s
9:	learn: 0.5666470	total: 1.02s	remaining: 19.5s
10:	learn: 0.5603600	total: 1.13s	remaining: 19.4s
11:	learn: 0.5513320	total: 1.23s	remaining: 19.3s
12:	learn: 0.5454433	total: 1.33s	remaining: 19.2s
13:	learn: 0.5392176	total: 1.43s	remaining: 19s
14:	learn: 0.5337498	total: 1.52s	remaining: 18.8s
15:	learn: 0.5283194	total: 1.63s	remaining: 18.7s
16:	learn: 0.5232887	total: 1.73s	remaining: 18.6s
17:	learn: 0.5181053	total: 1.83s	remaining: 18.6s
18:	learn: 0.5136784	total: 1.94s	remaining: 18.4s
19:	learn: 0.51

In [208]:
pY = (pY1 + 2*pY2 + 4*pY3 + 2*pY4) / 9

# pY = model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))
print(pY.sum() / pY.shape[0])

Local AUC ROC 0.8722553415985227
0.4364145333367825


In [209]:
X_t = np.array([pY1, pY2, pY3, pY4]).T

train_X_t, test_X_t, train_Y_t, test_Y_t = train_test_split(X_t, testY0, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)


p_t = model1.fit(train_X_t, train_Y_t.gender).predict_proba(test_X_t)[:, 1]

print('Local AUC ROC', roc_auc_score(test_Y_t.gender, p_t))

Local AUC ROC 0.8778387924611606
